In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd
from prophet import Prophet

In [ ]:
input_data = Path("data/")

In [ ]:
data = pd.read_csv("data/Waste_Collection___Diversion_Report__daily_.csv")
data['Report Date'] = data['Report Date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
data['Load Time'] = data['Load Time'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
data['Load Weight'] = data['Load Weight'].apply(lambda x: float(str(x).replace(',', '')))

In [ ]:
data.head()

In [ ]:
data[["Report Date", "Load Weight", "Load Type", "Route Number"]].head()

In [ ]:
data['Route Number'].value_counts()

In [ ]:
data[data["Route Number"] == "BU13"][["Report Date", "Load Weight", "Load Type", "Route Number"]].head()

In [ ]:
# data.groupby('Report Date')['Load Weight'].sum().plot()
df = data.groupby('Report Date')['Load Weight'].sum().reset_index()
df.columns = ['ds', 'y']
df.head()

In [ ]:
m = Prophet()
m.fit(df);
future = m.make_future_dataframe(periods=365);

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail();

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
data[data['Load Type'] == "GARBAGE COLLECTIONS"].groupby('Report Date')['Load Weight'].sum().plot();

In [ ]:
data[data['Load Type'] == "RECYCLING - SINGLE STREAM"].groupby('Report Date')['Load Weight'].sum().plot();

In [ ]:
data[data['Load Type'] == "SWEEPING"].groupby('Report Date')['Load Weight'].sum().plot();

In [ ]:
data.groupby('Report Date')['Load Weight'].sum().plot()

In [ ]:
# Adding regressors
clean_df = pd.read_csv(input_data / "clean_waste_data.csv")
clean_df['date'] = clean_df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# clean_df['Route.Type'].value_counts()
# clean_df.head()

# GARBAGE COLLECTION              201068
# RECYCLING - SINGLE STREAM        77246
# YARD TRIMMINGS                   45878
# BULK                              7898
# YARD TRIMMINGS-ORGANICS           6887

df = clean_df[clean_df['Route.Type'] == "GARBAGE COLLECTION"][['date', 'daily_weight']]
df = df.groupby('date')['daily_weight'].sum().reset_index()
df.columns = ['ds', 'y']
df.head()

In [ ]:
m = Prophet()
m.fit(df);
future = m.make_future_dataframe(periods=365);

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail();

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
df.dtypes